In [128]:
import tensorflow as tf
import os

In [129]:
#Limit gpu usage to avoid resource issues
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [130]:
import cv2
import imghdr
from matplotlib import pyplot as plt
import numpy as np

In [131]:
def filter_image_data(data_dir, image_exts):
    for image_class in os.listdir(data_dir):
        for image in os.listdir(os.path.join(data_dir, image_class)):
            image_path = os.path.join(data_dir, image_class, image)
            try:
                image = cv2.imread(image_path)
                tip = imghdr.what(image_path)
                if tip not in image_exts:
                    print('Image not in ext list {}'.format(image_path))
                    os.remove(image_path)
            except Exception as e:
                print('Issue with image {}'.format(image_path))

In [132]:
data_dir = 'images'
image_exts = ['jpeg', 'jpg', 'bmp', 'png'] #For filtering image data
filter_image_data(data_dir, image_exts)

In [133]:
data = tf.keras.utils.image_dataset_from_directory('images')
data_it = data.as_numpy_iterator()

Found 183 files belonging to 2 classes.


In [134]:
batch = data_it.next()
#Class 0 = cat
#Class 1 = dog

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [136]:
data = data.map(lambda x,y: (x/255, y))
scaled_it = data.as_numpy_iterator()
batch = scaled_it.next()
batch[0].max()

1.0

In [137]:
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.2)
test_size = int(len(data) * 0.1) + 1

if len(data) < train_size + val_size + test_size:
    train_size -= 1

In [138]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [139]:
from keras import Sequential
from keras import layers

In [140]:
#Creating the model
model = Sequential(
    [
        layers.Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)),
        layers.MaxPool2D(),
        layers.Conv2D(32, (3,3), 1, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, (3,3), 1, activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ]
)

In [141]:
#Compiling the model
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])
# model.summary()

In [142]:
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
epochs = 25
hist = model.fit(train, epochs=epochs, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [146]:
from keras import metrics

In [147]:
pre = metrics.Precision()
re = metrics.Recall()
acc = metrics.BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X,Y = batch
    yhat = model.predict(X)
    pre.update_state(Y, yhat)
    re.update_state(Y, yhat)
    acc.update_state(Y, yhat)

print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [ ]:
img = cv2.imread('cattest.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
np.expand_dims(resize, 0)
yhat = model.predict(np.expand_dims(resize/255, 0))

In [156]:
if yhat > 0.5:
    print(f'Predicted as a DOG')
else:
    print(f'Predicted as a CAT')

Predicted as a CAT


In [157]:
from keras import models

In [158]:
#Saving the model
model.save(os.path.join('model', 'cat-dog-classify_model.h5'))

c:\Dev\projects\deep-img-classifier\.venv\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
